In [17]:
%matplotlib inline
import numpy as np
import pandas as pd
import pandas_profiling
import matplotlib.pyplot as plt
import seaborn as sns
import re

In [2]:
cancun = pd.read_csv('../data/interim/cancun_unclean.csv')

In [3]:
# Initial report for exploratory
# profile = cancun.profile_report(title='Pandas Profiling Report')
# profile.to_file(output_file="cancun_unclean.html")

# Inital Exploration

In [4]:
cancun.shape

(6673, 37)

In [5]:
cancun.isnull().sum().sort_values(ascending=False).head(7)

localized_neighborhood    6673
star_rating               3044
response_time             1217
listing_name                 7
bathrooms                    2
bedrooms                     2
currency                     0
dtype: int64

## Dealing with Missing Values

Spider scraped 37 fields for around 6673 listings in Cancún but not all fields are important or have no missing values. That said, there is some manual selection to be done. Since there are only 37 fields, this is doable. If there were more (e.g. 100+ columns), manual selection would not be possible and we would have to adopt some rule system to remove low variance/high correlation/high NA columns.

`localized_neighborhood`, `star_rating`, and `response time` have a large number of NaN/NA values. `localized neighborhood` is instantly dropped as it's missing all its values; `star_rating` is substitutable with `avg_rating` and `response time` is not super crucial to our analysis and will also be dropped. There are 2 NAs for both `bedrooms` and `bathrooms` but we can simply drop those rows since there aren't a lot of them.

In [6]:
COLUMNS_TO_DROP = ['is_business_travel_ready', # Constant Column
                   'localized_neighborhood', # Not Important field
                   'amt_w_service', # Duplicate with price
                   'rate_type', # Constant Column
                   'star_rating', # 45% missing
                   'listing_name', # Not Important field
                   'response_time', # 18% missing + Not descriptive enough
                   'localized_city',
                   'host_id'] # Same with localized_neighborhood

We also need to separate the `VARIABLES` from the `FILTERS`. We want to create two DataFrames that are joinable by key.

In [7]:
FILTERS_WITH_URL = ['bathrooms', 'bedrooms', 'num_beds', 'can_instant_book',
                    'is_fully_refundable', 'is_superhost', 'is_new_listing',
                    'room_type_category', 'person_capacity', 'lat', 'lng', 
                     'url']

FILTERS = ['bathrooms', 'bedrooms', 'num_beds', 'can_instant_book',
           'is_fully_refundable', 'is_superhost', 'is_new_listing',
           'room_type_category', 'person_capacity', 'lat', 'lng']

## Filters

In [8]:
cancun_filters = (cancun
                  .drop(columns=COLUMNS_TO_DROP)
                  .dropna(how='any') 
                  .loc[:,FILTERS_WITH_URL] 
                 )                                            

In [9]:
cancun_filters.shape

(6669, 12)

In [10]:
cancun_filters.head(5)

,bathrooms,bedrooms,num_beds,can_instant_book,is_fully_refundable,is_superhost,is_new_listing,room_type_category,person_capacity,lat,lng,url
0,1.0,1.0,1,True,True,True,False,private_room,2,21.13566,-86.76741,https://www.airbnb.com/rooms/20776319
1,1.0,1.0,1,False,True,True,False,private_room,2,21.14571,-86.84190,https://www.airbnb.com/rooms/16492050
2,1.5,1.0,1,True,True,False,False,private_room,2,21.13119,-86.76394,https://www.airbnb.com/rooms/14266451
3,1.0,2.0,2,True,True,True,False,entire_home,5,21.15737,-86.83975,https://www.airbnb.com/rooms/17625889
4,1.0,1.0,2,True,True,False,False,entire_home,3,21.15327,-86.85548,https://www.airbnb.com/rooms/18543147


In [11]:
cancun_filters.columns

Index(['bathrooms', 'bedrooms', 'num_beds', 'can_instant_book',
       'is_fully_refundable', 'is_superhost', 'is_new_listing',
       'room_type_category', 'person_capacity', 'lat', 'lng', 'url'],
      dtype='object')

In [12]:
def exclude_columns(df, columns):
    """
    Exludes specified columns [List] from the DataFrame
    """
    return df[df.columns[~df.columns.isin(columns)]]

## Variables

In [13]:
cancun_variables = (cancun
                    .drop(columns=COLUMNS_TO_DROP)
                    .dropna(how='any') 
                    .pipe(exclude_columns, columns=FILTERS)
                   )

In [14]:
cancun_variables.shape

(6669, 17)

In [15]:
cancun_variables.head(5)

,accuracy,avg_rating,checkin,cleanliness,communication,currency,guest_satisfication,host_reviews,location,monthly_price_factor,picture_count,price,response_rate,reviews_count,url,value,weekly_price_factor
0,10,4.74,10,9,10,CAD,95,311,10,0.88,19,22,100,50,https://www.airbnb.com/rooms/20776319,10,0.88
1,10,4.93,10,10,10,CAD,99,147,10,0.85,22,28,100,108,https://www.airbnb.com/rooms/16492050,10,0.90
2,10,4.72,10,9,10,CAD,94,1376,9,1.00,14,30,100,243,https://www.airbnb.com/rooms/14266451,9,1.00
3,10,4.75,10,9,10,CAD,95,179,10,0.79,27,24,97,177,https://www.airbnb.com/rooms/17625889,10,0.93
4,10,4.65,10,9,10,CAD,93,631,9,0.80,18,21,100,194,https://www.airbnb.com/rooms/18543147,9,0.95


In [16]:
cancun_variables.columns

Index(['accuracy', 'avg_rating', 'checkin', 'cleanliness', 'communication',
       'currency', 'guest_satisfication', 'host_reviews', 'location',
       'monthly_price_factor', 'picture_count', 'price', 'response_rate',
       'reviews_count', 'url', 'value', 'weekly_price_factor'],
      dtype='object')

In [33]:
def extract_id(url):
    '''
    Helper funciton that extracts the listing id
    '''
    return re.search('/rooms/(.*)', url).group(1)

In [34]:
# Create new column listing_key
cancun_variables['listing_key'] = cancun_variables['url'].apply(extract_id)

In [36]:
# Do the same for cancun_filters
cancun_filters['listing_key'] = cancun_filters['url'].apply(extract_id)

In [37]:
cancun_filters.head()

,bathrooms,bedrooms,num_beds,can_instant_book,is_fully_refundable,is_superhost,is_new_listing,room_type_category,person_capacity,lat,lng,url,listing_key
0,1.0,1.0,1,True,True,True,False,private_room,2,21.13566,-86.76741,https://www.airbnb.com/rooms/20776319,20776319
1,1.0,1.0,1,False,True,True,False,private_room,2,21.14571,-86.84190,https://www.airbnb.com/rooms/16492050,16492050
2,1.5,1.0,1,True,True,False,False,private_room,2,21.13119,-86.76394,https://www.airbnb.com/rooms/14266451,14266451
3,1.0,2.0,2,True,True,True,False,entire_home,5,21.15737,-86.83975,https://www.airbnb.com/rooms/17625889,17625889
4,1.0,1.0,2,True,True,False,False,entire_home,3,21.15327,-86.85548,https://www.airbnb.com/rooms/18543147,18543147


In [38]:
# Export variables and filters

In [39]:
cancun_variables.to_csv('cancun_variables.csv', index=False)
cancun_filters.to_csv('cancun_filters.csv', index=False)

In [53]:
# profile = cancun_variables.profile_report(title='Pandas Profiling Report')
# profile.to_file(output_file="cancun_variables.html")


In [ ]:
# IDEA: We currently have the issue where we cannot benchmark our model. Perhaps to prove that it is better than random selection,
# scrape the 1st page of each price_range and see how much of our TOP 50 listings overlap with the 1st page Airbnb Listing Rankings,
# compare to a random selection of listings, bootstrap the proportion/median/difference and see whether we have a significance difference.

# Null Hypothesis: Our model will have the same overlap proportion with Airbnb Ranking (1st page) as a random selection
# Alternative Hypothesis: Our model will have more first page listings than a random selection

# This all depends on Airbnb's ranking algorithm. I wonder how it works! 

In [54]:
# TODO: Variable section within variables. Try to filter down to ~10 to work with